# Kaggle Leaf Classification Challenge


In [1]:
## Install pandas if you don't already have it (uncomment line below)
# ! pip install pandas

In [2]:
## If you are working on Colab, data_utils can be downloaded using the command below (uncomment line below)
# ! curl -O https://raw.githubusercontent.com/DeepLearningDTU/02456-2025/refs/heads/master/Week8%20Mini%20Project/data_utils.py

In [3]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

from IPython.display import clear_output
from skimage.io import imread
from skimage.transform import resize

import data_utils

#import sys
#sys.path.append(os.path.join('.', '..')) # Allow us to import shared custom 
#                                         # libraries, like utils.py

# Tying everything together

Now that you have learned about the most common network architectures, it is time to combine them into a more advanced model. 
It often happens that you have a combination of data that cannot easily be modeled by any single one of these types of network. Knowing how to divide the data into the right subsets, and then build a network that handles each subset efficiently can mean the difference between a great model and an unusable one. 

In this notebook, we will work on the **Kaggle Leaf Classification Challenge**, a data science competition from [`kaggle.com`](https://www.kaggle.com/) that contains several different kinds of data.
We will download the data, visualize it, and train a classifier.
A simple network with poor performance is provided for you as a starting point, but it is up to you use what you have learnt to improve the results.


## Kaggle challenge

Kaggle is a website to participate in real-world challenges.
Most competitions on Kaggle have a dataset, an accuracy metric and a leaderboard to compare submissions.
You can read more about Kaggle public datasets [here](https://www.kaggle.com/datasets).

We will undertake the [_Leaf Classification_](https://www.kaggle.com/c/leaf-classification) challenge. We report here the description of the dataset:

> The dataset consists of approximately 1,584 images of leaf specimens which have been converted to binary black leaves against white backgrounds. 
Three sets of features are also provided per image: a shape contiguous descriptor, an interior texture histogram, and a ﬁne-scale margin histogram. For each feature, a 64-attribute vector is given per leaf sample.


# Get set up

**NB**: You will need a Kaggle account for this exercise!

1. Go to [Kaggle](https://www.kaggle.com/), create an account
2. [Download the dataset](https://www.kaggle.com/c/leaf-classification/data)
3. Unpack the dataset in the current directory. Structure should be as follows:
```
02456-deep-learning-with-PyTorch/6_Mini_Project/leaf-classification
--> sample_submission.csv
--> test.csv
--> train.csv
--> images
--> --> 1.jpg
--> --> 2.jpg
--> --> 3.jpg
--> --> ...
```

# Upload data to colab
If you are running this notebook on Google Colab, you'll need to upload `data_utils.py` that we provide as well as the data you've just downloaded from Kaggle to Colab. Small files like `data_utils.py` you can simply upload via the folder icon to the left. Files that are uploaded like this, however, will be deleted every time the session ends. For big files like the kaggle data folder it's therefore better to:

1. Upload the data to Google Drive
2. Mount the Google Drive (see cell below)

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

After going through a quick authorization process you should now have access to the uploaded file via colab. We can check the location of our data as follows:

In [5]:
!ls drive/'My Drive'

ls: drive/My Drive: No such file or directory


Let's save the path for later use. Just replace this with your local path if you're not running on colab.

In [6]:
os.getcwd()

'/Users/carlahugod/Desktop/UNI/7sem/DL/02456-2025/Week8 Mini Project'

In [7]:
import sys

In [8]:
# add current directory to sys.path
sys.path.append('.')

In [9]:
if 'zhome' in os.getcwd().split('/'):
    path = '/zhome/7a/d/187526/Desktop/02456-2025/Week8 Mini Project/leaf-classification/'  # if you're working on Colab
else:
    path = '/Users/carlahugod/Desktop/UNI/7sem/DL/02456-2025/Week8 Mini Project/leaf-classification/'  # if you're folder structure is as defined above and you want to run locally
print("Using path:", path)

Using path: /Users/carlahugod/Desktop/UNI/7sem/DL/02456-2025/Week8 Mini Project/leaf-classification/


# Visualizing the data

First we start out by looking at the images. 
You need to load them first!
Then we load in the training data, which is in CSV format. For this, we use [pandas](https://pandas.pydata.org/).
Pandas is useful for data analysis, but we don't suggest using it in any production code.

In [10]:
image_paths = glob.glob(path+'images/*.jpg')
print("Total Observations:\t", len(image_paths))

# now loading the train.csv to find features for each training point
train = pd.read_csv(path + 'train.csv')
train_images = [path+'images/{}.jpg'.format(i) for i in train.id.values]

# now loading the test.csv
test = pd.read_csv(path + 'test.csv')

Total Observations:	 1584


## Exercise 1

1.1) How many samples do we have for training and test? Do we have the same information for training and test data? How many samples do we have for each species?

**Hint**: You might want to use .shape, .columns, pd.unique() and .symmetric_difference().


In [11]:
print('Train: {} training samples with {} features each.'.format(train.shape[0], train.shape[1]-2))
print('Test: {} test samples with {} features each.'.format(test.shape[0], test.shape[1]-2))
print("Unique species in train:", train['species'].nunique())


train.head()

Train: 990 training samples with 192 features each.
Test: 594 test samples with 191 features each.
Unique species in train: 99


,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [12]:
test.head()

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,4,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,...,0.006836,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711
1,7,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,...,0.000000,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922
2,9,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,...,0.128910,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000
3,12,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,...,0.012695,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789
4,13,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,...,0.000000,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812


 * **Answer:**

 We have 1584 training samples and 594 test samples. The training data has an additional column 'species' which is the target variable we want to predict. There are 99 unique species in the training data. The test data has an ID. 

With our training data and images loaded into memory.
It is time to take a look at the data.
Trying to classify leaves does not sound like a particularly difficult or interesting problem.
We have probably all had teachers forcing us to do it on field trips as children.

But try to take a look at **all** the different categories and come up with a system that discerns **all** types of leaves from each other. (In fact, distinguishing crops from weed using AI is already a thing: [weed-killing AI](https://www.cnbc.com/2018/06/04/weed-killing-ai-robot.html).)

In [13]:
# First we find an example of each species in order to visualize it
species = np.array(sorted(train.species.unique()))
species_examples = [np.random.choice(train[train.species == s].id.values) for s in species]

# Then we gather its index in our list of images in order to find the correct image
indexes = [image_paths.index(path + 'images/{}.jpg'.format(i)) for i in species_examples]

# Display the first image
plt.figure(figsize=(8, 8))
image = imread(image_paths[indexes[0]], as_gray=True)
plt.imshow(image, cmap='gray')
plt.title("%s" % (species[0]))
plt.axis('off')
plt.show()

## Exercise 2
2.1) In general it is a good approach to visualize an image for each category to get a better feeling of the task. You should now write some code to show 1 image from each category. You might want to use plt.subplot().

In [14]:
# Now plot 1 image from each category

plt.subplots(10, 10, figsize=(10, 10))
for i in range(train['species'].nunique()):
    plt.subplot(10, 10, i+1)
    image = imread(image_paths[indexes[i]], as_gray=True)
    plt.imshow(image, cmap='gray')
    plt.title("%s" % (species[i]))
    plt.axis('off')
plt.tight_layout()
plt.show()

As you can see, classifying leaves is actually a very tough problem.
What makes it even worse is that we cannot use all the image data we have available.
In order to decrease the amount of computation needed, we need to reduce the size of the images as much as possible.
On top of that, our neural network usually only accepts fixed-size input tensors.
This means we will have to change the shape of the images so that they all have the same sizes.


Resizing is problematic because it alters the shape of the leaves, and for some of them, this is their most distinctive feature. Take a look at `Salix_Intergra` in the bottom left corner.
Describing this leaf without taking its shape into account seems extremely difficult.

Therefore we will 
1. first pad all the images into squares, and
2. then resize them.

## Exercise 3

3.1) **Find an appropriate image size**. Test various resizings of the image until you have found the smallest resizing of the image where you "can still differentiate between the images".
How small is too small should ultimately be determined by an actual test, but what makes sense visually is probably a good place to start.
Change the `image_size = (?, ?)` parameter below, and note your choice.
 * **Answer:** (64, 64)


In [15]:
## Image pre-processing
image_size = (256,256)  # <-- YOUR CODE HERE TODO:

# Amount of images
amount = 5
image_sample = np.random.choice(train_images, amount)

# Define figure size
fig = plt.figure(figsize=(18, 6))

# Save original images in the figure
ax = plt.subplot(2, amount + 1, 1)
txt = ax.text(0.4, 0.5, 'Original', fontsize=20)
txt.set_clip_on(False)
plt.axis('off')
for i, path in enumerate(image_sample):
    plt.subplot(2, amount + 1, i + 2)
    image = imread(path, as_gray=True)    
    plt.imshow(image, cmap='gray')
    _id = int(path.split('/')[-1].split('.')[0])
    plt.title("{0}\nshape: {1}".format(train[train.id == _id].species.values[0], image.shape))
    plt.axis('off')
    
# Save resized images in the figure
ax = plt.subplot(2, amount + 1, len(image_sample) + 2)
txt = ax.text(0.4, 0.5, 'Resized', fontsize=20)
txt.set_clip_on(False)
plt.axis('off')
for i, path in enumerate(image_sample):
    i += len(image_sample) + 3
    plt.subplot(2, amount + 1, i)
    image = imread(path, as_gray=True)
    image = data_utils.pad2square(image)  # Make the image square
    image = resize(image, output_shape=image_size, mode='reflect', anti_aliasing=True)  # resizes the image
    plt.imshow(image, cmap='gray')

# Show plot
plt.show()

## Investigating the other features

Now that we have looked at the image data we have available, it is time to take a look at the other available features. Below we choose a random subset of the training data, and visualize the 3 types of available features:
* margin
* shape
* texture

Run it a few times to try and get an understanding of how the features differ from species to species.

In [16]:
# Extract features
X = train.values
species = X[:, 1:2]
margin = X[:, 2:66]
shape = X[:, 66:130]
texture = X[:, 130:]

# Let us plot some of the features
plt.figure(figsize=(21,7)) # Set the plot size
amount = 5                 # Choose the amount of images we want to show at a time

for i, idx in enumerate(np.random.choice(range(len(train)), amount)):
    ax = plt.subplot(amount,4,1+i*4)
    txt = ax.text(0.2, 0.2, species[idx][0], fontsize=20)
    txt.set_clip_on(False)
    plt.axis('off')
    if i == 0:
        plt.title('Species', fontsize=20)
    plt.subplot(amount,4,2+i*4)
    plt.plot(margin[idx])
    if i == 0:
        plt.title('Margin', fontsize=20)
    plt.axis('off')
    plt.subplot(amount,4,3+i*4)
    plt.plot(shape[idx])
    if i == 0:
        plt.title('Shape', fontsize=20)
    plt.axis('off')
    plt.subplot(amount,4,4+i*4)
    plt.plot(texture[idx])
    if i == 0:
        plt.title('Texture', fontsize=20)
    plt.axis('off')

plt.tight_layout()
plt.show()

## Exercise 4
So far we have learned about feed forward neural networks (FFNN), convolutional neural networks (CNN), recurrent neural networks (RNN), and transformers.

4.1) How could the `image`, `Margin`, `Shape` and `Texture` be used for classification, i.e. what kind of network type would you use for each of them, and why?
 * **Answer:**
 
In general CNN is good for images. FFNN is good for tabular data, whereas RNNs and transformers are good for sequential data.
Sequential data vs tabular data: Sequential data has an inherent order and temporal dependencies, while tabular data consists of independent rows and columns without a specific sequence. Time series is sequential. 

 * For the image data we would use a CNN since it is well suited for image data due to its ability to capture spatial hierarchies in images through the use of convolutional layers.
 * For the Shape data, which are structured numerical data, a FFNN would be appropriate as it can effectively model the relationships between the features and the target variable.
 * For the Margin and Texture data, which is sequential in nature, an RNN or transformer would be suitable as they are designed to handle sequential data and can capture temporal dependencies.



# Managing the data

The details of the code in this section isn't that important.
It simply manages the data in a nice way - so it is a good place to come back and look for inspiration when you will work on your own projects.


## Defining the data loader

## Exercise 5 
5.1) As a part of the data loader, we should specify the shape of the images, number of classes, and the number of features for the three feature types: margin, shape and texture. Define the three variables in the code below.

In [ ]:
# The image shape should be of the format (height, width, channels)
IMAGE_SHAPE =  (image.shape[0], image.shape[1], 1)  # <-- Your answer here TODO:
NUM_CLASSES =  train['species'].nunique()  # <-- Your answer here TODO:

# For all three features types margin, shape, and texture, we have NUM_FEATURES for each type.
NUM_FEATURES =  64  # <-- Your answer here TODO:

In [18]:
if 'zhome' in os.getcwd().split('/'):
    path = '/zhome/7a/d/187526/Desktop/02456-2025/Week8 Mini Project/leaf-classification/'  # if you're working on Colab
else:
    path = '/Users/carlahugod/Desktop/UNI/7sem/DL/02456-2025/Week8 Mini Project/leaf-classification/'  # if you're folder structure is as defined above and you want to run locally
print("Using path:", path)

Using path: /Users/carlahugod/Desktop/UNI/7sem/DL/02456-2025/Week8 Mini Project/leaf-classification/


In [19]:
# # Paths to data
# TRAIN_PATH = path + "train.csv" 
# TEST_PATH = path + "test.csv" 
# IMAGE_PATHS = glob.glob(path + "images/*.jpg")

# # train holds both X (input) and t (target/truth)
# data = data_utils.load_data(train_path=TRAIN_PATH, 
#                             test_path=TEST_PATH,
#                             image_paths=IMAGE_PATHS,
#                             image_shape=IMAGE_SHAPE[:2])
# # to visualize the size of the dimensions of the data
# print("\n@@@Shape checking of data sets@@@")
# print("TRAIN")
# print("\timages\t%s%f" % (data.train['images'].shape, data.train['images'].mean()))
# print("\tmargins\t%s\t%f" % (data.train['margins'].shape, data.train['margins'].mean()))
# print("\tshapes\t%s\t%f" % (data.train['shapes'].shape, data.train['shapes'].mean()))
# print("\ttextures%s\t%f" % (data.train['textures'].shape, data.train['textures'].mean()))
# print("\tts\t %s" % (data.train['ts'].shape))
# print("\twhile training, batch_generator will onehot encode ts to (batch_size, num_classes)")
# print("TEST")
# print("\timages\t%s\t%f" % (data.test['images'].shape, data.test['images'].mean())) 
# print("\tmargins\t%s\t%f" % (data.test['margins'].shape, data.test['margins'].mean()))
# print("\tshapes\t%s\t%f" % (data.test['shapes'].shape, data.test['shapes'].mean()))
# print("\ttextures%s\t%f" % (data.test['textures'].shape, data.test['textures'].mean()))
# print("\tids\t%s" % (data.test['ids'].shape))

#### Do you want to load your data quickly next time?
In the above cell the data is loaded, reshaped and stored as an object. If you want load this object quickly next time you should go for pickle. Pickle does effectively write an object into a character stream, so it can be loaded fastly next time.

In [20]:
# import pickle

# # # Dump the data into a pickle file
# with open(path + '/data.pickle', 'wb') as f:
#     pickle.dump(data, f)

In [21]:
# # Load the data from a pickle file
import pickle
with open(path + '/data.pickle', 'rb') as f:
    data = pickle.load(f)

## Batch Generator

While training, we will not directly access the entire dataset, instead we have a `batch_generator` function to give us inputs aligned with their targets/ids in a size that our model can handle in memory (batch\_size).

Furthermore, the `batch_generator` also handles validation splitting.

## Exercise 6
6.1) Explain shortly why the size of batches is important. You should comment on how the size of batches affect the memory, training speed, and the estimates of the gradients.
Choose a reasonable batch size the `batch_size = ?` parameter below, and note your choice.

 * **Answer:** 32
 The batch size affects memory, because larger batches will have larger gradient tensor that will need to be stored during training. Smaller batches will use less memory, but may lead to noisier gradient estimates and are more susceptible to outliers. Training speed can be faster with larger batches due to parallel processing, but very large batches can lead to diminishing returns. The estimates of the gradients are more accurate with larger batches, as they average over more samples, but this can also lead to less frequent updates and potentially slower convergence. 

In [22]:
batch_size = 128
dummy_batch_gen = data_utils.batch_generator(data, batch_size=batch_size, num_classes=NUM_CLASSES, num_iterations=5e3, seed=42)
train_batch = next(dummy_batch_gen.gen_train())
valid_batch, i = next(dummy_batch_gen.gen_valid())
test_batch, i = next(dummy_batch_gen.gen_test())

print("TRAIN")
print("\timages,", train_batch['images'].shape)
print("\tmargins,", train_batch['margins'].shape)
print("\tshapes,", train_batch['shapes'].shape)
print("\ttextures,", train_batch['textures'].shape)
print("\tts,", train_batch['ts'].shape)
print()
print("VALID")
print("\timages,", valid_batch['images'].shape)
print("\tmargins,", valid_batch['margins'].shape)
print("\tshapes,", valid_batch['shapes'].shape)
print("\ttextures,", valid_batch['textures'].shape)
print("\tts,", valid_batch['ts'].shape)
print()
print("TEST")
print("\timages,", test_batch['images'].shape)
print("\tmargins,", test_batch['margins'].shape)
print("\tshapes,", test_batch['shapes'].shape)
print("\ttextures,", test_batch['textures'].shape)
print("\tids,", len(test_batch['ids']))

TRAIN
	images, (128, 64, 64, 1)
	margins, (128, 64)
	shapes, (128, 64)
	textures, (128, 64)
	ts, (128, 99)

VALID
	images, (99, 64, 64, 1)
	margins, (99, 64)
	shapes, (99, 64)
	textures, (99, 64)
	ts, (99, 99)

TEST
	images, (128, 64, 64, 1)
	margins, (128, 64)
	shapes, (128, 64)
	textures, (128, 64)
	ids, 128


# Build the model

In [23]:
# Load functions
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

In [24]:
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")


def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x


def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

No GPU available.


## Exercise 7
7.1)  Now you must define the network architecture. It is always a good idea to start simple. We recommend you to start with a mix of a convolutional layer (maybe followed by max pooling layer), a recurrent layer, and a linear output layer such that we use all the features.  

When you build the model you should be aware of the dimensions of the input and output for your different layers. The function`permute` and `view` will be very useful to rearrange your dimensions.

In [25]:
import torch.nn.functional as F

height, width, channels = IMAGE_SHAPE

conv_out_channels = 128 # <-- Filters in your convolutional layer TODO
kernel_size = 3       # <-- Kernel size TODO
conv_stride = 1      # <-- Stride TODO
conv_pad    = 1      # <-- Padding TODO

# Keep track of features to output layer
rnn_hidden_size = 12  # GRU hidden size used to produce features_rnn TODO

# We treat each of the NUM_FEATURES as a time step with input_size=1 for the GRU
features_cat_size = conv_out_channels + 2 * NUM_FEATURES + rnn_hidden_size  # <-- Number of features concatenated before output layer
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Simple conv block
        self.conv_1 = Conv2d(in_channels=channels,
                             out_channels=conv_out_channels,
                             kernel_size=kernel_size,
                             stride=conv_stride,
                             padding=conv_pad)
        self.bn2 = nn.BatchNorm2d(conv_out_channels)
        self.pool = MaxPool2d(2)
        # make spatial dims small so features_img has size `conv_out_channels`
        self.adapt_pool = nn.AdaptiveAvgPool2d((1,1))

        self.dropout = nn.Dropout(0.5)

        self.bn1 = nn.BatchNorm1d(features_cat_size)
        
        # GRU: input_size=1 because we'll feed (batch, seq_len=NUM_FEATURES, 1)
        self.rnn_1 = GRU(input_size=1,
                         hidden_size=rnn_hidden_size,
                         batch_first=True)

        self.l_out = Linear(in_features=features_cat_size,
                            out_features=NUM_CLASSES,
                            bias=False)
    
        self.fc1 = Linear(in_features=features_cat_size,
                            out_features=features_cat_size,
                            bias=True)
        
    def forward(self, x_img, x_margin, x_shape, x_texture):
        features = []
        out = {}
        
        ## Convolutional layer ##
        # - Change dimensions to fit the convolutional layer 
        # - Apply Conv2d
        # - Use an activation function
        # - Reduce spatial dims and pool to (1,1) so features_img has size conv_out_channels
        x_img = x_img.permute(0, 3, 1, 2)  # (N, C, H, W)
        x_img = self.conv_1(x_img)
        x_img = self.bn2(x_img)
        x_img = relu(x_img)
        x_img = self.pool(x_img)
        x_img = self.adapt_pool(x_img)  # (N, conv_out_channels, 1, 1)
        features_img = x_img.view(x_img.size(0), -1)  # (N, conv_out_channels)
        features_img = self.dropout(features_img)

        # Append features to the list "features"
        features.append(features_img)

        
        ## Use concatenated leaf features for FFNN ##
        x = torch.cat((x_margin, x_texture), dim=1)  # (N, 2 * NUM_FEATURES)
        features.append(x)
        
        ## Use concatenated leaf features for RNN ##
        # - Change dimensions to fit GRU: (batch, seq_len=NUM_FEATURES, 1)
        # - Apply GRU and take final hidden state as feature vector
        x_rnn = x_shape.unsqueeze(2)  # (N, NUM_FEATURES, 1)
        _, h_n = self.rnn_1(x_rnn)
        features_rnn = h_n[-1]  # (N, rnn_hidden_size)
        features_rnn = self.dropout(features_rnn)
        features.append(features_rnn)

        # Use fully connected layer
        ## Fully connected layer where all features are in use ##
        features_fc = torch.cat(features, dim=1)
        features_fc = self.fc1(features_fc)
        features_fc = F.relu(features_fc)
        features_fc = self.bn1(features_fc)
        features_fc = self.dropout(features_fc)

        ## Output layer where all features are in use ##
        features_final = features_fc # was torch.cat(features, dim=1)
        
        out['out'] = self.l_out(features_final) 
        return out

net = Net()
if use_cuda:
    net.cuda()
print(net)

Net(
  (conv_1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (adapt_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (dropout): Dropout(p=0.5, inplace=False)
  (bn1): BatchNorm1d(268, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (rnn_1): GRU(1, 12, batch_first=True)
  (l_out): Linear(in_features=268, out_features=99, bias=False)
  (fc1): Linear(in_features=268, out_features=268, bias=True)
)


# Build the cost function

## Exercise 8
8.1) Since this is a classification task we will use the cross-entropy loss. Define the cross-entropy loss as the loss function in the code below.

In [26]:
LEARNING_RATE = 0.001 # TODO add all parameters to hyperparameter tuning
criterion = nn.CrossEntropyLoss()        # <-- Your code here.   

# weight_decay is equal to L2 regularization
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.8) # TODO: find optimal step size

print(f"Parameters to learn: {sum(p.numel() for p in net.parameters() if p.requires_grad)}")


def accuracy(ys, ts):
    predictions = torch.max(ys, 1)[1]
    correct_prediction = torch.eq(predictions, ts)
    return torch.mean(correct_prediction.float())

Parameters to learn: 101236


## Test network

#### Debugging 
The following cell might cause you some errors - try these suggestions before you try anyting else.

* **Your kernel keeps dying** on the line below it is most likely because you run out of memory.
The two most likely solutions are 
 1. reduce the image size further
 1. change your network architecture such that it uses less resources

* **`RuntimeError: size mismatch, m1: [??? x ???], m2: [??? x ???]`** 
 1. `features_cat_size` must match the actual output of the network i.e. the second dimension in `m1`.

* **Training is very slow**. This is most likely caused by the images. 
 1. Try and reduce the size of the images further, or reduce the dimensions of the network using either pooling or strides.


In [27]:
_img_shape = tuple([batch_size] + list(IMAGE_SHAPE))
_feature_shape = (batch_size, NUM_FEATURES)

def randnorm(size):
    return np.random.normal(0, 1, size).astype('float32')

# dummy data
_x_image = get_variable(Variable(torch.from_numpy(randnorm(_img_shape))))
_x_margin = get_variable(Variable(torch.from_numpy(randnorm(_feature_shape))))
_x_shape = get_variable(Variable(torch.from_numpy(randnorm(_feature_shape))))
_x_texture = get_variable(Variable(torch.from_numpy(randnorm(_feature_shape))))

# test the forward pass
output = net(x_img=_x_image, x_margin=_x_margin, x_shape=_x_shape, x_texture=_x_texture)
output['out']

tensor([[-1.0590, -0.6977, -1.3069,  ...,  0.8607, -0.9025, -0.3545],
        [-0.0501, -1.5022,  0.2858,  ...,  1.3588,  0.1029,  0.6751],
        [ 0.0710, -0.1558, -0.0777,  ...,  1.1356, -2.4320, -0.2594],
        ...,
        [ 1.0944,  0.0683,  0.7552,  ...,  0.6662, -0.2277, -0.2533],
        [-0.5993, -1.0248, -0.9121,  ..., -0.8967, -0.7760, -1.7360],
        [-1.7401,  1.1264,  0.7602,  ..., -0.2634, -0.4264,  0.1546]],
       grad_fn=<MmBackward0>)

# Train

In [28]:
import wandb
from tqdm import tqdm

os.environ['WANDB_API_KEY'] = os.getenv('WANDB_API_KEY')


/Users/carlahugod/Desktop/UNI/7sem/DL/02456-2025/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/Users/carlahugod/Desktop/UNI/7sem/DL/02456-2025/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or b

In [ ]:

# Setup settings for training 
VALIDATION_SIZE = 0.1 # 0.1 is ~ 100 samples for validation
max_iter = 3000 # Doesnt change much after 3k steps in my experience.
log_every = 100
eval_every = 100

# Function to get label
def get_labels(batch):
    return get_variable(Variable(torch.from_numpy(batch['ts']).long()))

# Function to get input
def get_input(batch):
    return {
        'x_img': get_variable(Variable(torch.from_numpy(batch['images']))),
        'x_margin': get_variable(Variable(torch.from_numpy(batch['margins']))),
        'x_shape': get_variable(Variable(torch.from_numpy(batch['shapes']))),
        'x_texture': get_variable(Variable(torch.from_numpy(batch['textures'])))
    }

# Initialize lists for training and validation
train_iter = []
train_loss, train_accs = [], []
valid_iter = []
valid_loss, valid_accs = [], []

# Generate batches
batch_gen = data_utils.batch_generator(data,
                                       batch_size=batch_size,
                                       num_classes=NUM_CLASSES,
                                       num_iterations=max_iter,
                                       seed=42,
                                       val_size=VALIDATION_SIZE)

# Log to WandB:
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="carlahugod-danmarks-tekniske-universitet-dtu",
    # Set the wandb project where this run will be logged.
    project="Week8_MiniProject_Leaf_Classification",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": LEARNING_RATE,
        "architecture": "Conv + GRU",
        "epochs": max_iter,
        "batch_size": batch_size,
        "image_size": IMAGE_SHAPE,
        "num_classes": NUM_CLASSES,
        "num_features": NUM_FEATURES,
        "conv_out_channels": conv_out_channels,
        "rnn_hidden_size": rnn_hidden_size,
        "kernel_size": kernel_size,
        "conv_stride": conv_stride,
        "conv_pad": conv_pad,
        "step_size": scheduler.step_size,
        "gamma": scheduler.gamma,
        "weight_decay": optimizer.param_groups[0]['weight_decay'],
        "validation_size": VALIDATION_SIZE,
    },
)


# Train network
net.train()
for i, batch_train in enumerate(batch_gen.gen_train()):
    if i % eval_every == 0:
        
        # Do the validaiton
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0
        for batch_valid, num in batch_gen.gen_valid():
            output = net(**get_input(batch_valid))
            labels_argmax = torch.max(get_labels(batch_valid), 1)[1]
            val_losses += criterion(output['out'], labels_argmax) * num
            val_accs += accuracy(output['out'], labels_argmax) * num
            val_lengths += num

        # Divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        valid_loss.append(get_numpy(val_losses))
        valid_accs.append(get_numpy(val_accs))
        valid_iter.append(i)
#         print("Valid, it: {} loss: {:.2f} accs: {:.2f}\n".format(i, valid_loss[-1], valid_accs[-1]))
        net.train()
    
    # Train network
    output = net(**get_input(batch_train))
    labels_argmax = torch.max(get_labels(batch_train), 1)[1]
    batch_loss = criterion(output['out'], labels_argmax)
    
    train_iter.append(i)
    train_loss.append(float(get_numpy(batch_loss)))
    train_accs.append(float(get_numpy(accuracy(output['out'], labels_argmax))))
    
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    scheduler.step() # Added lr scheduler
    run.log({"lr": scheduler.get_last_lr()[0]})
    
    # Log i figure
    if i % log_every == 0:
        run.log({"acc": valid_accs[-1], "loss": valid_loss[-1]})
        print("Train, it: {} loss: {:.2f} accs: {:.2f}".format(i, train_loss[-1], train_accs[-1]))
        print("Validation, it: {} loss: {:.2f} accs: {:.2f}".format(i, valid_loss[-1], valid_accs[-1]))
        fig = plt.figure(figsize=(12,4))
        plt.subplot(1, 2, 1)
        plt.plot(train_iter, train_loss, label='train_loss')
        plt.plot(valid_iter, valid_loss, label='valid_loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(train_iter, train_accs, label='train_accs')
        plt.plot(valid_iter, valid_accs, label='valid_accs')
        plt.legend()
        plt.show()
        clear_output(wait=True)

    if max_iter < i:
        run.finish()
        break

KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x290d40050>> (for post_run_cell), with arguments args (<ExecutionResult object at 29dda3470, execution_count=29 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 290c7aed0, raw_cell="
# Setup settings for training 
VALIDATION_SIZE = .." transformed_cell="# Setup settings for training 
VALIDATION_SIZE = 0.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/carlahugod/Desktop/UNI/7sem/DL/02456-2025/Week8%20Mini%20Project/8.1%20Kaggle%20Leaf%20Challenge.ipynb#Y415sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: Connection lost

In [ ]:
# Print all network parameters here. ie conv_out_channels, rnn_hidden_size, learning rate at the end,
#  bath sixe, image size, kernel size, stride, padding, model arcitecture

print(f"conv_out_channels: {conv_out_channels}")
print(f"rnn_hidden_size: {rnn_hidden_size}")
print(f"learning rate: {LEARNING_RATE}")
print(f"batch size: {batch_size}")
print(f"image size: {IMAGE_SHAPE}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Number of features: {NUM_FEATURES}")
print(f"kernel size: {kernel_size}")
print(f"stride: {conv_stride}")
print(f"padding: {conv_pad}")
print(f"Number of iterations: {max_iter}")
print(f"step size: {scheduler.step_size}")
print(f"gamma: {scheduler.gamma}")
print(f"weight decay (L2 regularization): {optimizer.param_groups[0]['weight_decay']}")
print(f"Validation size: {VALIDATION_SIZE}")

print(f"model architecture: {net}")
print(f"final validation loss: {valid_loss[-1]}")
print(f"final validation accuracy: {valid_accs[-1]}")


conv_out_channels: 128
rnn_hidden_size: 12
learning rate: 0.001
batch size: 256
image size: (128, 128, 1)
Number of classes: 99
Number of features: 64
kernel size: 3
stride: 1
padding: 1
Number of iterations: 3000
step size: 100
gamma: 0.8
weight decay (L2 regularization): 0
Validation size: 0.1
model architecture: Net(
  (conv_1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (adapt_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (dropout): Dropout(p=0.5, inplace=False)
  (bn1): BatchNorm1d(268, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (rnn_1): GRU(1, 12, batch_first=True)
  (l_out): Linear(in_features=268, out_features=99, bias=False)
  (fc1): Linear(in_features=268, out_features=268, bias=True)
)
final validation loss: 0.0871608555316925
final validation accuracy: 0

**TODO:**
1. Optimize parameters using hyperparameter tuning (e.g. learning rate, batch size, image size, conv_out_channels, rnn_hidden_size, number of layers, dropout, weight decay etc.)
2. try suggestions below
3. Add neptune logging to run on HPC
4. Implement config to add checkpoints and early stopping
5. Impkement easy config in the beginning of training

## Exercise 9

**Tip** This is a very small dataset (number of observations) compared to the number of features.
This means that overfitting may be an issue, and sometimes fancy tricks won't do any good. 
Keep that in mind, and always start simple.

**9.1) Improve the network**, and get as high a validation score as you can. 
When trying to improve the network nothing is sacred. You can try various learning rates, batch sizes, validation sizes, etc. 
And most importantly, the validation set is very small (only 1 sample per class), etc.

To get you off to a good start we have created a list of **things you might want to try**:
* Add more layers (mostly fully connected and convolutional)
* Increase or decrease the batch size 
* Use dropout (a lot - e.g. between the convolutional layers)
* Use batch normalization (a lot)
* Try with L2 regularization (weight decay)
* Use only the image for training (with CNN) - comment on the increased time between iterations.
* Change the image size to be bigger or smaller
* Try other combinations of FFN, CNN, RNN parts in various ways (bigger is not always better)

If your network is not performing as well as you would like it to, [here](http://theorangeduck.com/page/neural-network-not-working) is a great explanation of what might have gone wrong.


**9.2) Improve Kaggle score**. Once happy try to get the best score on Kaggle for this dataset as you can (**upload** instructions below)
You can upload your solution multiple times as you progress.
A very good implementation would get a score between $0.04$ to $0.06$ (the smaller the better), try and see if you can get there, and explain what might have gone wrong if you can't. 


**9.3) Reflect on the process**, and how you got to your final design and discuss your final results. 
What worked, and what didn't?
Include at least the following: 
* Description of the final architecture
* Description of the training parameters
* Description of the final results (Kaggle and validation)

**Answer:**


# Submission to Kaggle

First we have to make test set predictions, then we have to place the output in the submission file and then upload to Kaggle to get our score! You can upload up to 5 submissions per day.

In [ ]:
# GET PREDICTIONS
# containers to collect ids and predictions
ids_test, preds_test = [], []
net.eval()
# run like with validation
for batch_test, num in batch_gen.gen_test():
    output = net(**get_input(batch_test))
    y_out = output['out'].data

    ids_test += batch_test['ids']
    if num!=len(y_out):
        # in case of the last batch, num will be less than batch_size
        y_out = y_out[:num]
    preds_test.append(y_out)
preds_test = np.concatenate(preds_test, axis=0)
assert len(ids_test) == len(preds_test)

## Make submission file

In [ ]:
preds_df = pd.DataFrame(preds_test, columns=data.le.classes_)
ids_test_df = pd.DataFrame(ids_test, columns=["id"])
submission = pd.concat([ids_test_df, preds_df], axis=1)
submission.to_csv(drive_path + 'submission.csv', index=False)

# below prints the submission, can be removed and replaced with code block below
submission.head(5)

NameError: name 'drive_path' is not defined

## Upload submission

1. Go to [`https://www.kaggle.com/c/leaf-classification/submit`](https://www.kaggle.com/c/leaf-classification/submit)
3. Click or drop your submission here (writing a description is good practice)
4. Submit and look at where you are on the leaderboard.

Success! 